In [ ]:
!pip3 install transformers>=4.32.0 optimum>=1.12.0
!pip3 install auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu117/

Looking in indexes: https://pypi.org/simple, https://huggingface.github.io/autogptq-index/whl/cu117/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 14.2 MB/s eta 0:00:00


In [ ]:
import warnings
from transformers import logging

# Disable specific warnings from the transformers library
warnings.filterwarnings('ignore', message='The model .* is not supported for text-generation')

# Alternatively, you can disable all warnings
warnings.simplefilter('ignore')

# If you also want to disable transformers' specific logging, you can do this:
logging.set_verbosity_error() # This will show only error messages and suppress warnings


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model_name_or_path = "TheBloke/Llama-2-13B-chat-GPTQ"
# To use a different branch, change revision
# For example: revision="main"
model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
                                             device_map="auto",
                                             trust_remote_code=False,
                                             revision="main")

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

config.json:   0%|          | 0.00/837 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/7.26G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

In [ ]:
system_message = """I'm here to assist you with information about various monuments. Feel free to ask about any of the following landmarks: Ajanta Caves, Charar-E-Sharif, Chhota Imambara, Ellora Caves, Fatehpur Sikiri, Hawa Mahal, Gateway of India, Khajuraho, Sun Temple Konark, Alai Darwaza, Alai Minar, Basilica of Bom Jesus, Charminar, Golden Temple, Iron Pillar, Jamali Kamali Tomb, Lotus Temple, Mysore Palace, Qutub Minar, Taj Mahal, Tanjavur Temple, Victoria Memorial. Simply mention the name of the monument and ask about it."""
user_message_1 = "Hello Assistant"

In [ ]:
prompt_template=f'''[INST] <<SYS>>
{system_message}
<</SYS>>

{user_message_1} [/INST]'''

In [ ]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    temperature=0.7,
    top_p=0.95,
    repetition_penalty=1.15
)

raw_response = pipe(prompt_template)
response = raw_response[0]

In [ ]:
response

{'generated_text': "[INST] <<SYS>>\nI'm here to assist you with information about various monuments. Feel free to ask about any of the following landmarks: Ajanta Caves, Charar-E-Sharif, Chhota Imambara, Ellora Caves, Fatehpur Sikiri, Hawa Mahal, Gateway of India, Khajuraho, Sun Temple Konark, Alai Darwaza, Alai Minar, Basilica of Bom Jesus, Charminar, Golden Temple, Iron Pillar, Jamali Kamali Tomb, Lotus Temple, Mysore Palace, Qutub Minar, Taj Mahal, Tanjavur Temple, Victoria Memorial. Simply mention the name of the monument and ask about it.\n<</SYS>>\n\nHello Assistant [/INST]  Hello! I'm here to help answer your questions about various monuments in India. What would you like to know? Do you have a specific monument in mind that you'd like to learn more about? Please feel free to ask!"}

In [ ]:
model_reply_1 = response["generated_text"].split('[/INST]')[-1].lstrip()

In [ ]:
model_reply_1

"Hello! I'm here to help answer your questions about various monuments in India. What would you like to know? Do you have a specific monument in mind that you'd like to learn more about? Please feel free to ask!"

In [ ]:
user_message_2 = "Tell me about the history of Taj Mahal?"

In [ ]:
prompt_template=f'''[INST] <<SYS>>
{system_message}
<</SYS>>

{user_message_1} [/INST] {model_reply_1}</s><s>[INST] {user_message_2} [/INST]'''

In [ ]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    temperature=0.7,
    top_p=0.95,
    repetition_penalty=1.15
)

raw_response = pipe(prompt_template)
response = raw_response[0]

In [ ]:
model_reply_2 = response["generated_text"].split('[/INST]')[-1].lstrip()

In [ ]:
model_reply_2

"Certainly! The Taj Mahal is one of the most iconic and recognizable landmarks in India, and its history is a fascinating tale of love, loss, and architectural genius.\n\nThe Taj Mahal was built in the 17th century by Mughal Emperor Shah Jahan as a mausoleum for his beloved wife, Mumtaz Mahal, who died during childbirth in 1631. The emperor was so grief-stricken by her death that he ordered the construction of the Taj Mahal as a tribute to her memory.\n\nThe building of the Taj Mahal took over 20 years to complete, with thousands of artisans and laborers working around the clock to create this stunning masterpiece. It was constructed using white marble brought from Makrana, a town in Rajasthan, and features intricate inlay work of precious stones, including jasper, jade, and turquoise.\n\nThe Taj Mahal is not only a beautiful tomb but also a testament to the enduring power of love. It stands as a symbol of the Mughal Empire's grandeur and the emperor's devotion to his wife. Today, it i

In [ ]:
def check_token_limit(prompt_template: str, output_max_tokens: int, max_tokens:int=4000) -> bool:
    input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids
    total_tokens = input_ids.shape[1] + output_max_tokens

    if total_tokens > max_tokens:
        print(f"Total tokens ({total_tokens}) exceed the maximum limit ({max_tokens}).")
        return False
    else:
         # print(f"Total tokens ({total_tokens}) are within the limit ({max_tokens}).")
        return True


def build_prompt_within_limit(messages, model, tokenizer, max_new_tokens=512):
    system_message = """I'm here to assist you with information about various monuments. Feel free to ask about any of the following landmarks: Ajanta Caves, Charar-E-Sharif, Chhota Imambara, Ellora Caves, Fatehpur Sikiri, Hawa Mahal, Gateway of India, Khajuraho, Sun Temple Konark, Alai Darwaza, Alai Minar, Basilica of Bom Jesus, Charminar, Golden Temple, Iron Pillar, Jamali Kamali Tomb, Lotus Temple, Mysore Palace, Qutub Minar, Taj Mahal, Tanjavur Temple, Victoria Memorial. Simply mention the name of the monument and ask about it."""
    prompt_template = f'''[INST] <<SYS>>
{system_message}
<</SYS>>'''

    # Iterate through the messages in reverse, adding them in pairs
    for idx in range(len(messages) - 1, -1, -2):
        if idx == 0:
            # If this is the first message, just add the user content
            prompt_template += f'\n{messages[idx]["content"]} [/INST]'
        else:
            assistant_reply = messages[idx - 1]['content']
            user_message = messages[idx]['content']
            temp_prompt = f'{prompt_template}\n{assistant_reply}</s><s>[INST] {user_message} [/INST]'

            # Check if the temporary prompt is within the token limit
            if check_token_limit(temp_prompt, output_max_tokens=max_new_tokens):
                prompt_template = temp_prompt
            else:
                break

    return prompt_template



def continue_conversation(messages, model, tokenizer, max_new_tokens=512):
    system_message = """You are a helpful, respectful and honest chatbot."""
    prompt_template = f'''[INST] <<SYS>>
{system_message}
<</SYS>>'''

    # Add the initial user message
    prompt_template += f'\n{messages[0]["content"]} [/INST]'

    # Iterate through the remaining messages, adding them in pairs
    for idx in range(1, len(messages), 2):
        assistant_reply = messages[idx]['content']
        user_message = messages[idx + 1]['content'] if idx + 1 < len(messages) else ''
        prompt_template += f'\n{assistant_reply}</s><s>[INST] {user_message} [/INST]'

    limit_state = check_token_limit(prompt_template=prompt_template, output_max_tokens=max_new_tokens)
    if not limit_state:
        prompt_template = build_prompt_within_limit(messages, model, tokenizer, max_new_tokens)

    # Define the pipeline
    pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=max_new_tokens,
        temperature=0.7,
        top_p=0.95,
        repetition_penalty=1.15
    )

    # Generate a new response
    raw_response = pipe(prompt_template)
    response = raw_response[0]
    model_reply_new = response["generated_text"].split('[/INST]')[-1].lstrip()

    return model_reply_new


In [ ]:
import ipywidgets as widgets
from IPython.display import display
from IPython.core.display import HTML

In [ ]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [ ]:
# Define your function to respond to user input
def respond_to_user(sender):
    user_message_text = user_input.value
    user_message = [{"role": "user", "content": user_message_text}]
    response = continue_conversation(user_message, model, tokenizer)

    # Using <pre> to preserve formatting and different colors for user and chatbot
    user_html = f"<div style='color: #FF851B;'><b>You:</b><pre>{user_message_text}</pre></div><br>"
    response_html = f"<div style='color: #39CCCC;'><b>Chatbot response:</b><pre>{response}</pre></div><br>"

    display(HTML(user_html + response_html))
    user_input.value = '' # Clear the input box

# Create a text input widget
user_input = widgets.Text(
    placeholder='Ask something about monuments',
    description='You:',
)
display(user_input)

# Bind the respond_to_user function to the "Enter" key press event
user_input.on_submit(respond_to_user)


Text(value='', description='You:', placeholder='Ask something about monuments')